# Look below main() for Q3

In [47]:
import numpy as np
import pandas as pd
import random

In [48]:
def objfun(s):
    
    """ 
    This function calculates the objective function value

    Parameters: 
        s (array): Candidate solution in 1x10 vector form [s1,s2,s3,s3,s5,s6,s7,s8,s9,s10]

    Returns: 
        z (int): objective function value
    """
    
    z = 8*s[0] + 12*s[1] + 9*s[2] + 14*s[3] + 16*s[4] + 10*s[5] + 6*s[6] + 7*s[7] + 11*s[8] + 13*s[9]
    
    return z    

In [49]:
def check_feasibility(s):
    
    """ 
    This function checks whether the candidate solution is feasible according to given constraint the objective function value

    Parameters: 
        s (array): Candidate solution in 1x10 vector form [s1,s2,s3,s3,s5,s6,s7,s8,s9,s10]

    Returns: 
        constraint_flag (bool): The flag that indicates whether or not the candidate solution is feasible. If it is, then constraint_flag = True, else False
    """
    
    # Calculate constraint value
    const = 3*s[0] + 2*s[1] + s[2] + 4*s[3] + 3*s[4] + 3*s[5] + s[6] + 2*s[7] + 2*s[8] + 5*s[9]
    
    # Check whether the candidate solution is valid subject to problem constraint, return True is yes and False if no
    if const > 12:
        constraint_flag = False
    else:
        constraint_flag = True
    
    return constraint_flag   

In [50]:
def generate_rand_init_cond(n):
    
    """ 
    This function generates a random initial condition of vector size 1xn, choosing either 0 or 1 randomly

    Parameters: 
        n (int): Length of candidate solution, equal to the number of decision variables

    Returns: 
        s (array): Initial candidate solution of size 1xn
    """
    
    # Check whether the initial condition satisfies the problem constraint(s) and is feasible
    # Generate new initial conditions until problem constraint(s) are satisfied
    flag = False
    while flag == False:
        s = np.random.choice([0, 1], size=(n,))
        flag = check_feasibility(s)
    
    return s

In [51]:
def intial_cond():

    """ 
    This function allows the user to determine whether they would prefer to randomly generate an initial solution or enter their own

    Parameters: None

    Returns: 
        s (array): Initial candidate solution of size 1xn
    """
    
    # Prompt user to indicate whether they would prefer a random or user-defined initial condition
    print('Random (R) or User defined (U) initial condition? (Enter R or U):')
    
    sol_input = input()

    if sol_input == 'R':

        # number of decision variables as input 
        n = int(input('Number of decision variables (Enter an integer):'))

        # Generate randome initial condition
        s = generate_rand_init_cond(n)

    elif sol_input == 'U':

        # creating an empty list for initial solution to be appended to
        s_list = []

        # number of decision variables as input 
        n = int(input('Number of decision variables (Enter an integer):')) 

        print('Enter each element of the initial condition')

        # Iterate over entire initial condition to generate initial condition array, appending each user input boolean value
        for i in range(0, n): 
            ele = int(input())
            s_list.append(ele)
            
        # Convert list to Numpy array for use in other functions
        s = np.array(s_list)

    else:
        print('Invalid selection')
        
    return s

In [52]:
def bit_comp(s,i):

    """ 
    This function performs a single-bit complement of a binary array at a given position (i)

    Parameters:
        s (array): current candidate solution of size 1xn
        i (int): position at which the bit should be complemented

    Returns: 
        s_new (array): candidate solution after single-bit complement
    """
    
    s_new = np.copy(s)
    if s_new[i] == 0: 
        s_new[i] = 1
    else:
        s_new[i] = 0
    
    return s_new

In [53]:
def format_dataframe(data_store,k):

    """ 
    This function formats the output dataframe from the local search for presentation in a report

    Parameters:
        data_store (Pandas DatFrame): record of all data from the local search with the following columns: ['t', 's(t)', 'z', 'Neighbour', 'Bit', 'New_z']

    Returns: 
        data_store_formatted (Pandas DatFrame):record of all data from the local search in the correct format
    """
    
    # Iterate over all records
    for i in range(k):
    
        # Access the Neighbour array and assign to a
        a = data_store.loc[i,['Neighbour']][0]

        # Convert array to string in preparation for groupby
        data_store.loc[i,['Neighbour']] = np.array2string(a, separator=', ')

        # Access the s(t) array and assign to b
        b = data_store.loc[i,['s(t)']][0]

        # Convert array to string in preparation for groupby
        data_store.loc[i,['s(t)']] = np.array2string(b, separator=', ')
    
    # Perform groupby operation to generate the desired output format
    data_store_formatted = data_store.groupby(['t', 'z','s(t)', 'Neighbour','Bit','New_z'], as_index=True, sort=False).sum()    
    
    return data_store_formatted

In [ ]:
def main():

    # Generate initial condition based on random generation or user input 
    s = intial_cond()

    # Declare global iteration counter representing the index of the dataframe over all search iterations (used to address and append to dataframe locations)
    # Declare search iteration counter
    # Declare stop flag
    k = 0
    t = 1
    stop_flag = False

    # Create dataframe to store search data
    data_store = pd.DataFrame(columns=['t', 's(t)', 'z', 'Neighbour', 'Bit', 'New_z'])

    # Continue search iterations while the stopping criteria has not been reached
    while stop_flag == False:

        # Create dataframe to store current iteration objective function values and neighbours
        z_list = []

        # Calculate current objective function value
        z = objfun(s)

        # Perform single-bit complement for each bit in candidate solution, iterating over each bit in the solution vector
        # Check objective function value and store in dataframe (data_store) together with the corresponding bit position 
        for i in range(len(s)):

            # Store current data best solution data in dataframe
            data_store.loc[k,'t'] = t
            data_store.loc[k,'s(t)'] = s
            data_store.loc[k,'z'] = z

            # Perform bit complement on current bit position
            s_new = bit_comp(s,i)

            # Calculate objective function value
            z_new = objfun(s_new)

            # Check solution feasibility
            feasibility_flag = check_feasibility(s_new)

            # Set z_new to 'Infeasible' if candidate solution does not satisfy problem constraints
            if feasibility_flag == False:
                z_new = 'Infeasible'
            else:
                z_list.append(z_new)

            # Store new neighbour and objective function value in dataframe
            data_store.loc[k,'Neighbour'] = s_new
            data_store.loc[k,'Bit'] = i
            data_store.loc[k,'New_z'] = z_new

            # Increase global iteration counter
            k = k + 1

        # If the current iteration produces no better objective function value, stopping condition has been met. Set stop_flag = True
        if max(z_list) <= z:
            stop_flag = True

        # Else set new best solution to solution with best improvement
        else:

            # Isolate instances within current iteration only
            current_iter_data = data_store[data_store['t'] == t]

            # Convert New_z column to numeric in order to ignore 'Infeasible' strings when finding best solution index
            s_index = pd.to_numeric(current_iter_data['New_z'], errors='coerce').idxmax()

            # s now becomes the best solution of this iteration
            s = data_store.loc[s_index,['Neighbour']][0]

        # Increase search iteration counter
        t = t + 1

    # Format and return data_store DataFrame
    return format_dataframe(data_store,k)

## Question 3a

In [80]:
# Declare global iteration counter representing the index of the dataframe over all search iterations (used to address and append to dataframe locations)
# Declare search iteration counter
# Declare stop flag
k = 0
t = 1
stop_flag = False

s = intial_cond()
initial_s = s
z_list = []
z = objfun(s)
data_store = pd.DataFrame(columns=['t', 's(t)', 'z', 'Neighbour', 'Bit', 'New_z'])

Random (R) or User defined (U) initial condition? (Enter R or U):


 R
Number of decision variables (Enter an integer): 10


In [81]:
random_walk = pd.DataFrame(columns=['Solution', 'z'])
m = 100

for j in range(m):   
    
    neighbour_store = pd.DataFrame(columns=['Neighbour', 'Bit', 'z'])
    for i in range(len(s)):

            # Perform bit complement on current bit position
            s_new = bit_comp(s,i)

            # Calculate objective function value
            z_new = objfun(s_new)

            # Check solution feasibility
            feasibility_flag = check_feasibility(s_new)

            # Set z_new to 'Infeasible' if candidate solution does not satisfy problem constraints
            if feasibility_flag == False:
                z_new = 'Infeasible'
            else:
                z_list.append(z_new)

            # Store new neighbour and objective function value in dataframe
            neighbour_store.loc[k,'Neighbour'] = s_new
            neighbour_store.loc[k,'Bit'] = i
            neighbour_store.loc[k,'z'] = z_new

            # Increase global iteration counter
            k = k + 1
    
    valid_neighbours = neighbour_store[neighbour_store['z'] != 'Infeasible'].reset_index(drop=True)
    s_index = int(np.array(random.sample(range(len(valid_neighbours)), 1)))
    s = valid_neighbours.loc[s_index][0]
    
    random_walk.loc[j,'Solution'] = s
    random_walk.loc[j,'z'] = objfun(s_new)

In [85]:
random_walk.to_csv('random_walk.csv')

In [86]:
d = 1
f_bar = random_walk['z'].mean()
var = np.var(random_walk['z'])

r_1_term_1 = 1 / ((m - d)*var)

r_1_term_2 = 0
for i in range(m-d):
    
    r_1_term_2 = r_1_term_2 + (random_walk['z'][i] - f_bar)*(random_walk['z'][i+1] - f_bar)
    
r_1 = r_1_term_1*r_1_term_2
l = -1/np.log(abs(r_1))

print('r_1 = ', round(r_1,3))
print('l = ', round(l,3))

r_1 =  0.692
l =  2.717


#### Talbi pg 133 (PDF) Hamming distance

In [87]:
dist = []

for i in range(m):
    for j in range(m):
        sol1 = random_walk['Solution'][i]
        sol2 = random_walk['Solution'][j]
        
#         dist.append(np.sqrt(((sol1 - sol2)**2).sum()))
        dist.append(np.count_nonzero(sol1!=sol2))
        
diam_G = max(dist)
xi = l/diam_G
print('diam(G) = ', diam_G)
print('xi = ', round(xi, 3))

diam(G) =  9
xi =  0.302


## Question 3 b

In [88]:
sol_list = []
for j in range(10):
    for i in range(10):
        sol = [0,0,0,0,0,0,0,0,0,0]
        sol[j] = 1
        if i == j: continue
        sol[i] = 1
        
        if sol not in sol_list: sol_list.append(sol)
        
sol_list_45=[]
for i in range(len(sol_list)):
    sol_list_45.append(np.asarray(sol_list[i]))

In [89]:
def main2():
    
    data_store_main = pd.DataFrame(columns=['start_sol', 'start_z', 't', 'end_sol', 'end_z'])
    
    for j in range(len(sol_list_45)):

        # Generate initial condition based on random generation or user input 
        s = sol_list_45[j]

        # Declare global iteration counter representing the index of the dataframe over all search iterations (used to address and append to dataframe locations)
        # Declare search iteration counter
        # Declare stop flag
        k = 0
        t = 1
        stop_flag = False

        # Create dataframe to store search data

        data_store = pd.DataFrame(columns=['t', 's(t)', 'z', 'Neighbour', 'Bit', 'New_z'])
        
        data_store_main.loc[j,'start_sol'] = s
        data_store_main.loc[j,'start_z'] = objfun(s)

        # Continue search iterations while the stopping criteria has not been reached
        while stop_flag == False:

            # Create dataframe to store current iteration objective function values and neighbours
            z_list = []

            # Calculate current objective function value
            z = objfun(s)

            # Perform single-bit complement for each bit in candidate solution, iterating over each bit in the solution vector
            # Check objective function value and store in dataframe (data_store) together with the corresponding bit position 
            for i in range(len(s)):

                # Store current data best solution data in dataframe
                data_store.loc[k,'t'] = t
                data_store.loc[k,'s(t)'] = s
                data_store.loc[k,'z'] = z

                # Perform bit complement on current bit position
                s_new = bit_comp(s,i)

                # Calculate objective function value
                z_new = objfun(s_new)

                # Check solution feasibility
                feasibility_flag = check_feasibility(s_new)

                # Set z_new to 'Infeasible' if candidate solution does not satisfy problem constraints
                if feasibility_flag == False:
                    z_new = 'Infeasible'
                else:
                    z_list.append(z_new)

                # Store new neighbour and objective function value in dataframe
                data_store.loc[k,'Neighbour'] = s_new
                data_store.loc[k,'Bit'] = i
                data_store.loc[k,'New_z'] = z_new

                # Increase global iteration counter
                k = k + 1

            # If the current iteration produces no better objective function value, stopping condition has been met. Set stop_flag = True
            if max(z_list) <= z:
                stop_flag = True

            # Else set new best solution to solution with best improvement
            else:

                # Isolate instances within current iteration only
                current_iter_data = data_store[data_store['t'] == t]

                # Convert New_z column to numeric in order to ignore 'Infeasible' strings when finding best solution index
                s_index = pd.to_numeric(current_iter_data['New_z'], errors='coerce').idxmax()

                # s now becomes the best solution of this iteration
                s = data_store.loc[s_index,['Neighbour']][0]

            # Increase search iteration counter
            t = t + 1
            
        data_store_main.loc[j,'t'] = t
        data_store_main.loc[j,'end_sol'] = s
        data_store_main.loc[j,'end_z'] = objfun(s)

    # Format and return data_store DataFrame
    return data_store_main

In [90]:
data_store_main = main2()

In [96]:
data_store_main

,start_sol,start_z,t,end_sol,end_z
0,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0]",20,4,"[1, 1, 0, 1, 1, 0, 0, 0, 0, 0]",50
1,"[1, 0, 1, 0, 0, 0, 0, 0, 0, 0]",17,5,"[1, 0, 1, 1, 1, 0, 1, 0, 0, 0]",53
2,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0]",22,4,"[1, 1, 0, 1, 1, 0, 0, 0, 0, 0]",50
3,"[1, 0, 0, 0, 1, 0, 0, 0, 0, 0]",24,4,"[1, 1, 0, 1, 1, 0, 0, 0, 0, 0]",50
4,"[1, 0, 0, 0, 0, 1, 0, 0, 0, 0]",18,5,"[1, 1, 1, 0, 1, 1, 0, 0, 0, 0]",55
5,"[1, 0, 0, 0, 0, 0, 1, 0, 0, 0]",14,5,"[1, 0, 1, 1, 1, 0, 1, 0, 0, 0]",53
6,"[1, 0, 0, 0, 0, 0, 0, 1, 0, 0]",15,4,"[1, 0, 0, 1, 1, 0, 0, 1, 0, 0]",45
7,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0]",19,4,"[1, 0, 0, 1, 1, 0, 0, 0, 1, 0]",49
8,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 1]",21,4,"[1, 0, 1, 0, 1, 0, 0, 0, 0, 1]",46
9,"[0, 1, 1, 0, 0, 0, 0, 0, 0, 0]",21,5,"[0, 1, 1, 1, 1, 0, 0, 0, 1, 0]",62


### LLM(U)

In [91]:
LLM_U = data_store_main['t'].mean()
print('LLM(U) = ', round(LLM_U,3))

LLM(U) =  4.533


## Question 3 c

### Delta Amp

In [92]:
Amp_U_num = len(data_store_main['start_z'])*(max(data_store_main['start_z']) - min(data_store_main['start_z']))
Amp_U_den = data_store_main['start_z'].sum()
Amp_U = Amp_U_num/Amp_U_den
print("Amp(U) = ", round(Amp_U,3))

Amp(U) =  0.802


In [93]:
Amp_O_num = len(data_store_main['end_z'])*(max(data_store_main['end_z']) - min(data_store_main['end_z']))
Amp_O_den = data_store_main['end_z'].sum()
Amp_O = Amp_O_num/Amp_O_den
print("Amp(O) = ", round(Amp_O,3))

Amp(O) =  0.349


In [94]:
delta_Amp = (Amp_U - Amp_O) / Amp_U
print("Delta Amp = ", round(delta_Amp,3))

Delta Amp =  0.565


### Gap(O)

In [95]:
sum_delta_f = 0
for i in range(len(data_store_main)):
    sum_delta_f = sum_delta_f + data_store_main.loc[i,'end_z'] - max(data_store_main['end_z'])
    
Gap_O = sum_delta_f/(len(data_store_main)*max(data_store_main['end_z']))
print(sum_delta_f)
print("Gap(O) = ", round(Gap_O,3))

-339
Gap(O) =  -0.122
